# Calibrated 1980 initial conditions

For a fair comparison we need the right ocean layer temperatures to initialise FaIR with in 1980. So we re-run the constrained calibrated ensemble *with AR6 best estimate forcing* and save out results in 1980.

In [1]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

/nfs/b0110/Users/mencsm/miniconda3/envs/frida-forcing/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scenarios = ["AR6_hist"]  # doesn't matter which

In [3]:
df_configs = pd.read_csv('../data/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
df_configs.index.name = 'config'

In [4]:
f = FAIR()
f.define_time(1750, 1980, 1)
f.define_scenarios(scenarios)
f.define_configs(df_configs.index)
# Create an aggregate forcing time series specie.
f.define_species(['Fext'],
    {
        'Fext': {
            'type': 'unspecified',
            'input_mode': 'forcing',
            'greenhouse_gas': False,
            'aerosol_chemistry_from_emissions': False,
            'aerosol_chemistry_from_concentration': False
        }
    }
)
f.allocate()

In [5]:
# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(f.climate_configs["ocean_heat_transfer"], df_configs.loc[:, "kappa1":"kappa3"].values)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

In [6]:
df_ar6 = pd.read_csv('../data/AR6_ERF_1750-2019.csv', index_col=0)
df_ar6.loc[1750:1980, 'total']

year
1750    0.297568
1751    0.286642
1752    0.261955
1753    0.224661
1754    0.184225
          ...   
1976    0.493672
1977    0.662900
1978    0.784609
1979    0.842734
1980    0.856720
Name: total, Length: 231, dtype: float64

In [7]:
f.forcing[:, 0, :, 0] = df_ar6.loc[1750:1980, 'total'].values[:, None]

In [8]:
# initial conditions: 1750
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [9]:
f.run()

Running 1001 projections in parallel: 100%|██| 230/230 [00:00<00:00, 6295.74timesteps/s]


In [10]:
# rebase to 1850-1900 means
weight_51yr = np.ones(52)
weight_51yr[0] = 0.5
weight_51yr[-1] = 0.5

In [11]:
temp_rebased = f.temperature - np.average(f.temperature[100:152, ...], weights=weight_51yr, axis=0)

In [12]:
temp_rebased[-1, 0, :, :]

<xarray.DataArray (config: 1001, layer: 3)>
array([[0.28236657, 0.23423612, 0.18633487],
       [0.19743352, 0.11288936, 0.11123615],
       [0.1526687 , 0.26755832, 0.27895316],
       ...,
       [0.33675012, 0.15101247, 0.06793531],
       [0.31535483, 0.19977371, 0.16757373],
       [0.24759308, 0.24739948, 0.22241974]])
Coordinates:
    timebounds  float64 1.98e+03
    scenario    <U8 'AR6_hist'
  * config      (config) int64 223 248 337 1126 ... 1496559 1497149 1498501
  * layer       (layer) int64 0 1 2

In [13]:
temp_rebased[-1, 0, :, :].to_pandas()

layer,0,1,2
config,,,
223,0.282367,0.234236,0.186335
248,0.197434,0.112889,0.111236
337,0.152669,0.267558,0.278953
1126,0.187971,0.125814,0.083456
1483,0.510628,0.339496,0.067510
...,...,...,...
1492261,0.426868,0.369430,0.302564
1495153,0.349554,0.178557,0.105316
1496559,0.336750,0.151012,0.067935


In [14]:
temp_rebased[-1, 0, :, :].to_pandas().to_csv('../output/fair_1980_state.csv')